<a href="https://colab.research.google.com/github/Kennethfargose/Time-Series-Analysis/blob/main/univariate_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install darts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.9/693.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.6/720.6 kB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46

In [ ]:
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm

import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.models import Prophet
from darts.metrics import mape, mae, r2_score, smape,marre,mase,ope,rmsle,mse
from darts.dataprocessing.transformers import Scaler
import glob
import os
from darts.models import NBEATSModel, RNNModel,AutoARIMA,TransformerModel,BlockRNNModel

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def convert_multivariate_ts_to_list(ts_multivariate: TimeSeries):
    out_list = []
    for component in ts_multivariate.components:
        out_list.append(ts_multivariate.univariate_component(component))
    return out_list

In [ ]:

# # Set the directory path where the files are located
# directory = '/content/drive/MyDrive/TS'
# MODEL_KWARGS={}
# VALIDATION_START = pd.Timestamp("2015-04-30")

# # Process the CSV files and get the results
# autoarima = classical_model(
#     directory=directory,

#     model=AutoARIMA,
#     model_name="Autoarima",
#     validation_start_date=VALIDATION_START,
#     model_kwargs={
#         # 'country_holidays': "UnitedStates",
#         }

# )



In [ ]:
autoarima

NameError: ignored

In [ ]:
def convert_multivariate_ts_to_list(ts):
    return [ts.drop_variable(var).rename(var) for var in ts.variables()]

def deep_learning_models(directory, model_class, model_name, validation_start_date, model_kwargs):
    csv_files = [file for file in os.listdir(directory) if file.endswith('.csv')]
    transformer = Scaler()
    res = []

    # Process each CSV file
    for file in tqdm(csv_files):
        file_name = os.path.splitext(file)[0]
        file_path = os.path.join(directory, file)
        df_name = file_name.replace(" ", "_")
        df = pd.read_csv(file_path)

        if 'date' in df.columns:
            df.loc[:, df.columns != 'date'] = df.loc[:, df.columns != 'date'].mask(df.loc[:, df.columns != 'date'] < 100)
            df.interpolate(inplace=True)

        dfs = [df[['date', col]] for col in df.columns if col != 'date']

        for df_uni in dfs:
            name = df_uni.columns[1]
            ts_name = df_name + "_" + name + "_ts"
            ts = TimeSeries.from_dataframe(df_uni, time_col='date', value_cols=name)
            globals()[ts_name] = ts

            ts_train, ts_test = ts.split_after(validation_start_date)
            ts_train = transformer.fit_transform(ts_train)
            ts_test_scaled = transformer.transform(ts_test)

            ts_val_scaled = ts_test_scaled[:194]

            model = model_class(**model_kwargs)  # Initialize model for each univariate series
            new_model = model.fit(series=ts_train, val_series=ts_val_scaled)
            preds = transformer.inverse_transform(new_model.predict(n=60))

            res.append(pd.DataFrame(
                {
                "model_name": [model_name],
                "ts_name": [name],
                "r2": [r2_score(ts_test, preds)],
                "mae": [mae(ts_test, preds)],
                "mape": [mape(ts_test, preds)],
                "smape": [smape(ts_test, preds)],
                }
            ))

    return pd.concat(res)

directory = '/content/drive/MyDrive/TS'
MODEL_KWARGS = {
    "input_chunk_length": 14,
    'model': "LSTM",
    'optimizer_kwargs': None,
    "batch_size": 32,
    "n_epochs": 10,
    'n_rnn_layers': 1,
    'log_tensorboard': True,
    "model_name": "RNN_thesis",
    'force_reset': True,
    'dropout': 0,
    'training_length': 24 ,
    'show_warnings': True,
    "pl_trainer_kwargs": {
        "accelerator": "gpu",
        "devices": [0]
    }
}

VALIDATION_START = pd.Timestamp("2015-04-30")

RNN_1 = deep_learning_models(
    directory=directory,
    model_class=RNNModel,
    model_name="RNN_thesis",
    validation_start_date=VALIDATION_START,
    model_kwargs=MODEL_KWARGS
)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def convert_multivariate_ts_to_list(ts):
    return [ts.drop_variable(var).rename(var) for var in ts.variables()]

def deep_learning_models(directory, model, model_name,validation_start_date, model_kwargs):
    # Get a list of all CSV files in the directory
    csv_files = [file for file in os.listdir(directory) if file.endswith('.csv')]
    transformer=Scaler()
    # Create an empty list to store the results
    res = []
    model = model(**model_kwargs)
    model_name = model_name

    # Process each CSV file
    for file in tqdm(csv_files):  # Wrap csv_files with tqdm for progress bar
        file_name = os.path.splitext(file)[0]  # Extract the file name without extension
        file_path = os.path.join(directory, file)  # Construct the file path
        df_name = file_name.replace(" ", "_")  # Replace spaces in file name with underscores (optional)
        df = pd.read_csv(file_path)  # Read the CSV file into a DataFrame

        # Interpolate values in the DataFrame
        if 'date' in df.columns:
            df.loc[:, df.columns != 'date'] = df.loc[:, df.columns != 'date'].mask(df.loc[:, df.columns != 'date'] < 100)
            df.interpolate(inplace=True)

        # Split the multivariate dataframe into a list of univariate dataframes
        dfs = [df[['date', col]] for col in df.columns if col != 'date']

        # Process each univariate DataFrame
        for df_uni in dfs:
            name = df_uni.columns[1]  # Get the variable name

            # Convert DataFrame to TimeSeries and name it based on the variable name
            ts_name = df_name + "_" + name + "_ts"  # Construct the TimeSeries name
            ts = TimeSeries.from_dataframe(df_uni, time_col='date', value_cols=name)
            globals()[ts_name] = ts  # Assign TimeSeries to a variable with dynamic name

            ts_train, ts_test = ts.split_after(validation_start_date)
            ts_train = transformer.fit_transform(ts_train)
            ts_test_scaled = transformer.transform(ts_test)

            ts_val_scaled=ts_test_scaled[:194]
            New_model = model.fit(series=ts_train,val_series=ts_val_scaled)
            preds = transformer.inverse_transform(New_model.predict(n=60))
            res.append(pd.DataFrame(
                {
                "model_name": [model_name],
                "ts_name": [name],
                "r2": [r2_score(ts_test, preds)],
                "mae": [mae(ts_test, preds)],
                "mape": [mape(ts_test, preds)],
                "smape": [smape(ts_test, preds)],
                }
            ))

    # Create a DataFrame from the results
    return pd.concat(res)

In [ ]:
directory = '/content/drive/MyDrive/TS'
MODEL_KWARGS = {
    "input_chunk_length": 14,
    'model': "LSTM",
    'optimizer_kwargs': None,
    "batch_size": 32,
    "n_epochs": 10,
    'n_rnn_layers': 1,
    'log_tensorboard': True,
    "model_name": "RNN_thesis",
    'force_reset': True,
    'dropout': 0,
    'training_length': 24 ,
    'show_warnings': True
}

VALIDATION_START = pd.Timestamp("2015-04-30")

# Process the CSV files and get the results
RNN_1 = deep_learning_models(
    directory=directory,
    model=RNNModel,
    model_name="RNN_thesis",
    validation_start_date=VALIDATION_START,
    model_kwargs=MODEL_KWARGS
)

  0%|          | 0/17 [00:00<?, ?it/s]INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | LSTM             | 2.8 K 
4 | V             | Linear           | 26    
---------------------------------------------------
2.8 K     Trainable params
0         Non-trainable params
2.8 K     Total params
0.011     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (48) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower valu

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.trainer.connectors.signal_connector:[rank: 0] Received SIGTERM: 15
INFO:pytorch_lightning.trainer.connectors.signal_connector:[rank: 0] Received SIGTERM: 15
INFO:pytorch_lightning.trainer.connectors.signal_connector:[rank: 0] Received SIGTERM: 15
  6%|▌         | 1/17 [00:40<10:54, 40.92s/it]


SIGTERMException: ignored

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
RNN_1

In [ ]:
directory = '/content/drive/MyDrive/TS'
MODEL_KWARGS={}
VALIDATION_START = pd.Timestamp("2015-04-30")

# Process the CSV files and get the results
NB = deep_learning_models(
            directory=directory,
            model = NBEATSModel,
            model_name = "Nbeats",
            validation_start_date=VALIDATION_START,

            model_kwargs={"input_chunk_length": 14,
                  "output_chunk_length": 1,
                "generic_architecture":True,
                'num_stacks':30,
                "n_epochs":100,
              'activation':'ReLU',
                'dropout': 0,
                'layer_widths':256,
                'expansion_coefficient_dim':5,
                'trend_polynomial_degree':2,
                "force_reset": True,
                'num_blocks': 1,
                'num_layers': 4,
                'batch_size':32,

                'model_name':'Nbeats_final'

                  }


)

In [ ]:
NB